In [2]:
import pandas as pd 
import numpy as np
from scipy.sparse import csr_matrix 

# display results to 3 decimal points, not in scientific notstion

pd.set_option('display.float_format', lambda x :'%.3f' %x)

user_data = pd.read_table('C:/Users/vishu/Documents/data/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv',
                          header = None , nrows =2e7,
                          names = ['users','musicbrainz-artist-id','artist-name','plays'],
                          usecols = ['users','artist-name','plays'])

user_profile = pd.read_table('C:/Users/vishu/Documents/data/lastfm-dataset-360K/usersha1-profile.tsv',
                             header=None,
                             names =['users','gender','age','country','signup'],
                             usecols =['users','country']    )


In [4]:
user_data.head()

,users,artist-name,plays
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137.000
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099.000
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897.000
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717.000
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706.000


In [27]:
user_profile.head()

,users,country
0,00000c289a1829a808ac09c00daf10bc3c4e223b,Germany
1,00001411dc427966b17297bf4d69e7e193135d89,Canada
2,00004d2ac9316e22dc007ab2243d6fcb239e707d,Germany
3,000063d3fe1cf2ba248b9e3c3f0334845a27a6bf,Mexico
4,00007a47085b9aab8af55f52ec8846ac479ac4fe,United States


In [6]:
if user_data['artist-name'].isnull().sum() >0:
    user_data = user_data.dropna(axis=0, subset= ['artist-name'])

In [10]:
artist_plays = (user_data.groupby(by =['artist-name'])['plays'].
                sum().
                reset_index().
                rename(columns = {'plays': 'total_artist_plays'})
                [['artist-name','total_artist_plays']])

In [11]:
user_data.head()

,users,artist-name,plays
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137.000
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099.000
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897.000
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717.000
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706.000


In [13]:
artist_plays.head()

,artist-name,total_artist_plays
0,!!!,9365.000
1,!deladap,193.000
2,!distain,128.000
3,!t.o.o.h.!,101.000
4,#####,6626.000


In [16]:
user_data_with_artist_plays = user_data.merge(artist_plays, left_on = 'artist-name', right_on = 'artist-name', how= 'left')
user_data_with_artist_plays.head()

,users,artist-name,plays,total_artist_plays
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137.000,2326.000
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099.000,84730.000
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897.000,3744.000
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717.000,7528.000
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706.000,3912.000


In [18]:
 artist_plays['total_artist_plays'].describe()

count    52938.000
mean      1770.649
std      10812.297
min          1.000
25%         47.000
50%        172.000
75%        632.000
max     796800.000
Name: total_artist_plays, dtype: float64

In [61]:
print(artist_plays['total_artist_plays'].quantile(np.arrange(.9,1,.01))),

AttributeError: module 'numpy' has no attribute 'arrange'

In [23]:
popularity_threshold = 40000
user_data_popular_artists = user_data_with_artist_plays.query('total_artist_plays >= @popularity_threshold')

In [24]:
user_data_popular_artists.head()

,users,artist-name,plays,total_artist_plays
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099.000,84730.000
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691.000,304659.000
12,00000c289a1829a808ac09c00daf10bc3c4e223b,goldfrapp,361.000,64259.000
17,00000c289a1829a808ac09c00daf10bc3c4e223b,dropkick murphys,302.000,103921.000
24,00000c289a1829a808ac09c00daf10bc3c4e223b,jack johnson,227.000,162014.000


In [30]:
user_profile['country'].head()

0          Germany
1           Canada
2          Germany
3           Mexico
4    United States
Name: country, dtype: object

In [32]:
combined = user_data_popular_artists.merge(user_profile, left_on ='users', right_on = 'users',how = 'left')
usa_data = combined.query('country == \'United States\'')
usa_data.head()

,users,artist-name,plays,total_artist_plays,country
47,00007a47085b9aab8af55f52ec8846ac479ac4fe,devendra banhart,456.000,64370.000,United States
48,00007a47085b9aab8af55f52ec8846ac479ac4fe,boards of canada,407.000,157636.000,United States
49,00007a47085b9aab8af55f52ec8846ac479ac4fe,cocorosie,386.000,63865.000,United States
50,00007a47085b9aab8af55f52ec8846ac479ac4fe,aphex twin,213.000,106973.000,United States
51,00007a47085b9aab8af55f52ec8846ac479ac4fe,animal collective,203.000,84295.000,United States


In [69]:
if not usa_data[usa_data.duplicated(['users', 'artist-name'])].empty:
    initial_rows = usa_data.shape[0]

print('Initial dataframe shape {0}'.format(usa_data.shape))
usa_data = usa_data.drop_duplicates(['users', 'artist-name'])
current_rows = usa_data.shape[0]
print ('New dataframe shape {0}'.format(usa_data.shape))
#print ('Removed {0} rows'.format(initial_rows - current_rows))


Initial dataframe shape (27182, 5)
New dataframe shape (27182, 5)


In [70]:
wide_artist_data = usa_data.pivot(index = 'artist-name', columns = 'users' , values = 'plays').fillna(0)
wide_artist_data_sparse = csr_matrix(wide_artist_data.values)

In [71]:
from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric ='cosine', algorithm ='brute')
model_knn.fit(wide_artist_data_sparse)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius=1.0)

In [74]:
query_index = np.random.choice(wide_artist_data.shape[0])
distances, indices = model_knn.kneighbors(wide_artist_data.iloc[query_index, :].values.reshape(1,-1),n_neighbors = 6)

for i in range(0 ,len(distances.flatten())):
        if i == 0:
            print ('Recommendation for {0}:\n'.format(wide_artist_data.index[query_index]))
        else :
            print ('{0}: {1},with distance of {2}:'.format(i, wide_artist_data.index[indices.flatten()[i]]))


Recommendation for mindless self indulgence:



IndexError: tuple index out of range